In [1]:
# Home assignment from Wirecard.
# received 16.01 
# Notebook for prototyping

In [2]:
#Import packages
import pandas as pd
import numpy as np
import scipy as sp
from scipy.special import gamma
from scipy.special import gammaln as lnG
from scipy import optimize


import datetime as dt

# Task 1

In [3]:
#load the data
df = pd.read_csv("all_transactions.csv",names=['ID','date'])
print(df.shape)
df.head()

(6919, 2)


,ID,date
0,1,19970101
1,1,19970118
2,1,19970802
3,1,19971212
4,2,19970101


## Prepare the dataset

In [4]:
# Remove duplications
df_unique = df.drop_duplicates()
print('Number of unique samples: {}'.format(len(df_unique)))
print('Number of unique customers: {}'.format(len(df_unique.ID.drop_duplicates())))

Number of unique samples: 6696
Number of unique customers: 2357


In [5]:
# train-validation split
split_date_b = 19970101 # Jan 1997
split_date_e = 19971001 # Oct 1997
#query = 'date >= {} and date < {}'.format(split_date_b,split_date_e)
#query = (df_unique.date >= split_date_b) & df_unique.date < split_date_e
df_train = df_unique[(df_unique.date >= split_date_b) & (df_unique.date < split_date_e)] # training part
#df_train = df_unique.query(query) # training part
df_val = df_unique[df_unique.date >= split_date_e] # rest is our validation part
print('Training   data size: {}'.format(df_train.shape))
print('Validation data size: {}'.format(df_val.shape))

Training   data size: (4814, 2)
Validation data size: (1882, 2)


In [6]:
# save the .csv file
df_train.reset_index(drop=True).to_csv('cal_period_transactions.csv')

# Task 2

In [7]:
# read the .csv file 
df = pd.read_csv("cal_period_transactions.csv",index_col='Unnamed: 0')
df.shape

(4814, 2)

## Parameters calculation 

### Convert to a proper datetime format

In [8]:
df['datetime'] = pd.to_datetime(df['date'],format='%Y%m%d')

### x: number of transactions done by customer

In [9]:
df['x'] = df.groupby('ID')['ID'].transform(lambda s: s.count() - 1)
df.head()

,ID,date,datetime,x
0,1,19970101,1997-01-01,2
1,1,19970118,1997-01-18,2
2,1,19970802,1997-08-02,2
3,2,19970101,1997-01-01,1
4,2,19970113,1997-01-13,1


### tx: duration in weeks between customer's last and first transaction

In [10]:
# create temp df with first and last date columns
df_temp = df.datetime.groupby(df['ID']).agg(['first','last'])

In [11]:
# find the difference between the transactions 
df_temp['tx_days'] = (df_temp['last'] - df_temp['first'])
# convert it to weeks
df_temp['tx'] = df_temp['tx_days']/np.timedelta64(1,'W')
# round to 2 decimals as in the example
df_temp['tx'] = df_temp['tx'].round(2)

In [12]:
df_temp.head()

,first,last,tx_days,tx
ID,,,,
1,1997-01-01,1997-08-02,213 days,30.43
2,1997-01-01,1997-01-13,12 days,1.71
3,1997-01-01,1997-01-01,0 days,0.00
4,1997-01-01,1997-01-01,0 days,0.00
5,1997-01-01,1997-01-01,0 days,0.00


### T: Duration in weeks between end of calibration period and the first customer's transaction 

In [13]:
df_temp['T_days'] = dt.datetime.strptime(str(split_date_e), '%Y%m%d') - df_temp['first'] - np.timedelta64(1,'D')# -1 to take the last day into account

In [14]:
df_temp['T'] = (df_temp['T_days'] / np.timedelta64(1,'W')).round(2)

In [15]:
df_temp.head(5)

,first,last,tx_days,tx,T_days,T
ID,,,,,,
1,1997-01-01,1997-08-02,213 days,30.43,272 days,38.86
2,1997-01-01,1997-01-13,12 days,1.71,272 days,38.86
3,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86
4,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86
5,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86


### Merge into the initial DF

In [16]:
df = pd.merge(df,df_temp.drop(columns=['first','last','tx_days','T_days']),on=['ID'])

In [17]:
df.head()

,ID,date,datetime,x,tx,T
0,1,19970101,1997-01-01,2,30.43,38.86
1,1,19970118,1997-01-18,2,30.43,38.86
2,1,19970802,1997-08-02,2,30.43,38.86
3,2,19970101,1997-01-01,1,1.71,38.86
4,2,19970113,1997-01-13,1,1.71,38.86


### Prepare the output df

In [18]:
# only unique customers should be stored
df_out = df.drop_duplicates(subset=['ID'])

In [19]:
df_out.head()

,ID,date,datetime,x,tx,T
0,1,19970101,1997-01-01,2,30.43,38.86
3,2,19970101,1997-01-01,1,1.71,38.86
5,3,19970101,1997-01-01,0,0.00,38.86
6,4,19970101,1997-01-01,0,0.00,38.86
7,5,19970101,1997-01-01,0,0.00,38.86


In [20]:
# prepare the output df
df_out = df_out.reset_index().drop(columns=['date','datetime','index'])
# rename ID column
df_out.rename(columns={'ID': 'Customer ID'},inplace=True)
# Save into .csv file
df_out.to_csv('summary_customers.csv')

In [21]:
df_out.head()

,Customer ID,x,tx,T
0,1,2,30.43,38.86
1,2,1,1.71,38.86
2,3,0,0.00,38.86
3,4,0,0.00,38.86
4,5,0,0.00,38.86


In [22]:
# clean the memory
del df_temp

To be omplemented
* check whether dates are in chronological order, i.e. avoid negative values 

In [23]:
df_test = pd.read_csv('summary_customers.csv')

# Task 3

The aim of this assignment is to fit a model to the data, so that predictions about the customers’ purchase behavior in future can be made. The model is defined by four parameters namely r, α, a, b and they are always greater than 0. For fitting the model (i.e. finding the parameters r, α, a, b), Maximum Likelihood Estimation (MLE) can be used.

In [24]:
# load the .csv file
df = pd.read_csv('summary_customers.csv',index_col='Unnamed: 0')
df.head()

,Customer ID,x,tx,T
0,1,2,30.43,38.86
1,2,1,1.71,38.86
2,3,0,0.00,38.86
3,4,0,0.00,38.86
4,5,0,0.00,38.86


In [86]:
# Implementing in the form of class
class CustomFunction:
    def __init__(self, verbose=False):
        """Constructor + initialization of the fit parameters"""
        self.verbose = verbose
        #initialize model parameters
        self.__initialize_par()
        
    def __obj_f(self, params, df):
        """Implementation of the objective function.
        
        NLL = -1/N Sum( lnA1i + lnA2i + ln( exp(lnA3i) + delta exp(lnA4i)) )
        """
        #update parameters
        self.__update_params(params)
        
        #normalise alpha
        alpha_norm =self.alpha * self.norm
        # protection from negative parameters
        if self.r <=0 or self.b <=0 or self.a <= 0 or alpha_norm <= 0:
            return np.inf
        # number os samples
        N = len(df)
        # loss function
        obj_i = self.__loss(df,self.r,alpha_norm,self.a,self.b)
        # protection from the inf values that might arise from division by zero
        if obj_i.isin([np.inf]).values.any():
            return np.inf
        # normalised NLL
        out = obj_i.sum() * (-1) / N
        
        return out
    
    def __loss(self, df,r,alpha,a,b):
        """Implementation of the loss function.
        """
        lnA1 = lnG(r+df.x) + r*np.log(alpha) - lnG(r)
        lnA2 = lnG(a+b) + lnG(b+df.x) - lnG(b) - lnG(a+b+df.x)
        lnA3 = -(r+df.x)*np.log(alpha+df['T'])
        lnA4 = np.log(a) - np.log(b+df.x-1) - (r+df.x)*np.log(alpha + df.tx)
        deltaA4 = self.__deltaA4(df.x,np.exp(lnA4))
        out = lnA1 + lnA2 + np.log(np.exp(lnA3) + deltaA4)
        return out
    
    def __deltaA4(self,x,A4):
        """Implementation of the delta function.
        deltaX = 0 if x <= 0 
        deltaX = X otherwise
        """
        A4[x <= 0] = 0
        return A4
    
    def __update_params(self, new_params):
        """Update model parameters
        """
        self.r = new_params[0].copy()
        self.alpha = new_params[1].copy()
        self.a = new_params[2].copy()
        self.b = new_params[3].copy()
    
    def __initialize_par(self):
        """Method to initialize parameters with Gaus(1,0.05)
        """
        self.r, self.alpha, self.a, self.b = np.random.normal(1, 0.05,4)
    
    def set_r(self, r):
        """Method to set parameter r
        """        
        if r <= 0: 
            raise ValueError('r should be positive')
        self.r = r
        
    def set_alpha(self, alpha):
        """Method to set parameter alpha
        """
        if alpha <= 0: 
            raise ValueError('alpha should be positive')
        self.alpha = alpha
    
    def set_a(self, a):
        """Method to set parameter a
        """        
        if a <= 0: 
            raise ValueError('a should be positive')
        self.a = a
        
    def set_b(self, b):
        """Method to set parameter b
        """
        if b <= 0: 
            raise ValueError('b should be positive')
        self.b = b
    
    def set_parameters(self, r, alpha, a, b):
        """Method to set the fit parameters
        """
        self.set_r(r), self.set_alpha(alpha), self.set_a(a), self.set_b(b)
    
    def fit(self, df, minimizer_strategy = [(2,'Nelder-Mead')]):
        """Implementation of the custom minimisation procedure.
        
        Parameters:
        df - pandas DataFrame with the input data of the next format (x,tx,T),
        minimizer_strategy - list of pears specifying the minimzersand number of calls to be used
        (default [(2,'Nelder-Mead')])
        """
        # check the input data
        self.__validate_df(df)
        # compute the scalefactor and normalise the data
        norm_df = self.__normalise_data(df)
        # initial parameters
        initial_pars = [self.r,self.alpha,self.a,self.b]
    
        for num_iter,minimizer in minimizer_strategy:
            print('Run ',minimizer)
            for i in range(1,num_iter+1):
                print('Step ',i)
                initial_pars = [self.r,self.alpha,self.a,self.b]
                if self.verbose: print('Initialisation: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
                # perform the minimization
                self.fit_res = sp.optimize.minimize(self.__obj_f,x0 = initial_pars,args=(norm_df), method=minimizer) 
                # update the model parameters
                self.__update_params(self.fit_res.x)
                if i!= num_iter+1 and self.verbose:
                    self.print_final_results()
                print('\n')
                
    def fit_global(self, df, minimizer_strategy = 'Nelder-Mead',num_iter = 100):
        """Implementation of the global (brut-force) minima finding algorithm - basinhopping
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html#scipy.optimize.basinhopping
        Large number of iterations is recommended due to a step-based optimisation procedure.
        Fit succeed if 15% of num_iter the global minimum candidate remains the same.
        
        Parameters:
        df - pandas DataFrame with the input data of the next format (x,tx,T),
        minimizer_strategy - The minimization method (default: Nelder-Mead),
        num_iter - number of max iterations (default: 100)
        """
        # check the input data
        self.__validate_df(df)
        # compute the scalefactor and normalise the data
        norm_df = self.__normalise_data(df)
        # initial parameters
        initial_pars = [self.r,self.alpha,self.a,self.b]
        print('********Basin-hopping minimisation********')
        if self.verbose: print('Initialisation: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
        # perform the minimisation
        self.fit_res = sp.optimize.basinhopping(self.__obj_f,x0 = initial_pars, 
                                            minimizer_kwargs={"method": minimizer_strategy,'args':(norm_df)},
                                            stepsize = 0.05,
                                            niter = num_iter,
                                            niter_success = int(num_iter*0.15),
                                            disp = self.verbose)
        # update the model parameters
        self.__update_params(self.fit_res.x)
    
    def __validate_df(self,df):
        """Method to check whether the input DataFrame contains
        required Series (x,xt,T) and free of NaNs"""
        cols = ['x','xt','T']
        for c in cols:
            if c not in df.columns:
                raise ValueError('Column ' + c + ' is missing in the input DataFrame.')
    
    def __normalise_data(self,df):
        """Method to normalise parameters tx and T
        """
        df_temp = df.copy()
        self.__norm_sf(df)
        df_temp['tx']*=self.norm
        df_temp['T']*=self.norm
        return df_temp
    
    def __norm_sf(self,df):
        """Method to compute the normalisation factor
        """
        max_T = df['T'].max()
        self.norm = 10./ max_T
    
    def print_final_results(self):
        """Method to print final results
        """
        print('*********Minimisation is done*********')
        # For some minimzers success and status are not defined
        # e.g. doesn't work for basinhopping
        try:
            print('Success: ',self.fit_res.success)
            print('Status: ',self.fit_res.status)
        except Exception: pass
        print('NLL: ',self.fit_res.fun)
        print('Optimized parameters: r = ',self.r, ' alpha = ',self.alpha, ' a = ',self.a, ' b = ',self.b)
        if self.verbose:
            print(self.fit_res)
            
    def getFitResults(self):
        """Method to return the fit results
        """
        try:
            return self.fit_res
        except Exception:
            print('Fit has not been applied yet, no fit result object exist.')
            return None
    
    def getFitParameters(self):
        """Method to return the fit parameters
        """
        return self.r, self.alpha, self.a, self.b


In [87]:
model1 = CustomFunction(verbose=False)

Fit has not been applied yet, no fit result object exist.


NoneType

## Minimizers

### Nelder-Mead

the most robust optimizer in scipy. converges even from not very good starting values
Problems: takes a large number of iterations. Can get stuck in almost local minima or stops just short of minimum. Doesn't use gradient for convergence check.


### Newton

"newton" is a simple solver implemented in statsmodels. It includes now a ridge factor to reduce problems with near singular hessian.
works well for discrete models, few iterations, uses Hessian
problems: breaks if Hessian is not positive definite. Doesn't have line search and might break if gradient is very large at the starting values.

### BFGS

works well and fast if we are close enough to the minimum. Finds an optimum with very small gradient.
problems: can diverge or fail to converge if gradient is large, or shape of function is not "nice" during the optimization.

### basinhopping

Find the global minimum of a function using the basin-hopping algorithm

Basin-hopping is a two-phase method that combines a global stepping algorithm with local minimization at each step. Designed to mimic the natural process of energy minimization of clusters of atoms, it works well for similar problems with “funnel-like, but rugged” energy landscapes [5].

As the step-taking, step acceptance, and minimization methods are all customizable, this function can also be used to implement other two-phase methods.

In [74]:
#np.random.seed(1)

model = CustomFunction(verbose=True)
#model.fit(df,minimizer_strategy=[(2,'Nelder-Mead')])
model.fit_global(df,minimizer_strategy='Nelder-Mead',num_iter=300)
model.print_final_results()

********Basin-hopping minimisation********
Initialisation: r =  100  alpha =  40  a =  20  b =  30


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in log


basinhopping step 0: f 2.65055
basinhopping step 1: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
found new global minimum on step 1 with function value 2.65055
basinhopping step 2: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
found new global minimum on step 2 with function value 2.65055
basinhopping step 3: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
basinhopping step 4: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
basinhopping step 5: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
found new global minimum on step 5 with function value 2.65055
basinhopping step 6: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
found new global minimum on step 6 with function value 2.65055
basinhopping step 7: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
basinhopping step 8: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
basinhopping step 9: f 2.65055 trial_f 2.65055 accepted 1  lowest_f 2.65055
basinhopping step 10: f 2.65055 t